In [42]:
from pyspark.sql import  SQLContext, Row
from pyspark import SparkContext,SparkConf
import pandas as pd
from pyspark.sql.functions import regexp_extract, regexp_replace, when,udf,col,count,sum,avg,round
from pyspark.sql.types import DoubleType,IntegerType,StringType
import pyspark.sql.functions as F

import time

In [2]:
sc = SparkContext('local')
sqlCtx = SQLContext( sc )

In [3]:
df1 = sqlCtx.read.csv(r'c:\data\2014년 졸음운전 교통사고.csv', 
                     header = True, inferSchema = True, encoding='euc-kr' )
df2 = sqlCtx.read.csv(r'c:\data\2015년 졸음운전 교통사고.csv', 
                     header = True, inferSchema = True, encoding='euc-kr' )
df3 = sqlCtx.read.csv(r'c:\data\2016년 졸음운전 교통사고.csv', 
                     header = True, inferSchema = True, encoding='euc-kr' )

### 1. 3개의 데이터를 병합하시요.

In [4]:
df_all = df1.unionAll(df2)
df_all = df_all.unionAll(df3)

In [5]:
df_all.show(5)

+---------+--------+--------+--------+
|     구분|사고(건)|사망(명)|부상(명)|
+---------+--------+--------+--------+
|2014년1월|     169|       7|     347|
|2014년2월|     145|       3|     268|
|2014년3월|     206|      12|     358|
|2014년4월|     182|       4|     360|
|2014년5월|     218|      17|     474|
+---------+--------+--------+--------+
only showing top 5 rows



In [6]:
df_all.printSchema()

root
 |-- 구분: string (nullable = true)
 |-- 사고(건): integer (nullable = true)
 |-- 사망(명): integer (nullable = true)
 |-- 부상(명): integer (nullable = true)



In [7]:
df_all = df_all.withColumn('date', F.to_timestamp('구분', format='yyyy년MM월'))
df_all.show(5)

+---------+--------+--------+--------+-------------------+
|     구분|사고(건)|사망(명)|부상(명)|               date|
+---------+--------+--------+--------+-------------------+
|2014년1월|     169|       7|     347|2014-01-01 00:00:00|
|2014년2월|     145|       3|     268|2014-02-01 00:00:00|
|2014년3월|     206|      12|     358|2014-03-01 00:00:00|
|2014년4월|     182|       4|     360|2014-04-01 00:00:00|
|2014년5월|     218|      17|     474|2014-05-01 00:00:00|
+---------+--------+--------+--------+-------------------+
only showing top 5 rows



### 2. 년도 및 월별 평균 사망자를 보여주시요

In [46]:
df_all.groupBy( F.year( 'date').alias('year'), F.month( 'date').alias('month') )\
            .mean('사망(명)').orderBy('year','month').show()

+----+-----+-------------+
|year|month|avg(사망(명))|
+----+-----+-------------+
|2014|    1|          7.0|
|2014|    2|          3.0|
|2014|    3|         12.0|
|2014|    4|          4.0|
|2014|    5|         17.0|
|2014|    6|         13.0|
|2014|    7|         14.0|
|2014|    8|          9.0|
|2014|    9|         15.0|
|2014|   10|         14.0|
|2014|   11|         10.0|
|2014|   12|         12.0|
|2015|    1|          8.0|
|2015|    2|          8.0|
|2015|    3|          4.0|
|2015|    4|         10.0|
|2015|    5|          9.0|
|2015|    6|          8.0|
|2015|    7|          7.0|
|2015|    8|         19.0|
+----+-----+-------------+
only showing top 20 rows



### 3. 2015년 1월 ~ 7월 데이터를 보여주시요.

In [15]:
df_all.where( (df_all['date'] >= '2015-01') & (df_all['date'] <= '2015-08')).show()

+---------+--------+--------+--------+-------------------+
|     구분|사고(건)|사망(명)|부상(명)|               date|
+---------+--------+--------+--------+-------------------+
|2015년1월|     205|       8|     375|2015-01-01 00:00:00|
|2015년2월|     175|       8|     375|2015-02-01 00:00:00|
|2015년3월|     237|       4|     492|2015-03-01 00:00:00|
|2015년4월|     227|      10|     486|2015-04-01 00:00:00|
|2015년5월|     231|       9|     504|2015-05-01 00:00:00|
|2015년6월|     207|       8|     401|2015-06-01 00:00:00|
|2015년7월|     273|       7|     556|2015-07-01 00:00:00|
+---------+--------+--------+--------+-------------------+



In [40]:
df_all.where(f"`date` between '2015-01' and '2015-08'").show()

+---------+--------+--------+--------+-------------------+----+
|     구분|사고(건)|사망(명)|부상(명)|               date|year|
+---------+--------+--------+--------+-------------------+----+
|2015년1월|     205|       8|     375|2015-01-01 00:00:00|2015|
|2015년2월|     175|       8|     375|2015-02-01 00:00:00|2015|
|2015년3월|     237|       4|     492|2015-03-01 00:00:00|2015|
|2015년4월|     227|      10|     486|2015-04-01 00:00:00|2015|
|2015년5월|     231|       9|     504|2015-05-01 00:00:00|2015|
|2015년6월|     207|       8|     401|2015-06-01 00:00:00|2015|
|2015년7월|     273|       7|     556|2015-07-01 00:00:00|2015|
+---------+--------+--------+--------+-------------------+----+



### 4.  2016년 사고대비 사망율을 구하시요
- sql을 사용하는것보다 데이터프레임 함수를 사용하는것이 더 좋을수도 있음

In [16]:
df_all = df_all.withColumn('year' ,F.year('date'))

In [49]:
df_2015 = df_all.where((df_all['year'] == '2015'))

In [17]:
df_2016 = df_all.where((df_all['year'] == '2016'))

In [18]:
df_2016.createOrReplaceTempView('df_2016')

In [44]:
sql = 'select `구분`, `사고(건)`, `사망(명)`,  round((`사망(명)`/`사고(건)`),2) \
        as `사고대비 사망율` from df_2016'

start = time.time()
sqlDF = sqlCtx.sql( sql )
sqlDF.show()
end = time.time()
print(end-start)

+----------+--------+--------+---------------+
|      구분|사고(건)|사망(명)|사고대비 사망율|
+----------+--------+--------+---------------+
| 2016년1월|     192|       5|           0.03|
| 2016년2월|     174|       6|           0.03|
| 2016년3월|     217|       7|           0.03|
| 2016년4월|     216|       7|           0.03|
| 2016년5월|     239|      13|           0.05|
| 2016년6월|     200|      12|           0.06|
| 2016년7월|     227|       9|           0.04|
| 2016년8월|     230|       7|           0.03|
| 2016년9월|     187|      13|           0.07|
|2016년10월|     183|      10|           0.05|
|2016년11월|     200|       5|           0.03|
|2016년12월|     168|       4|           0.02|
+----------+--------+--------+---------------+

0.1621870994567871


In [63]:
start = time.time()
df_all.where("`구분`>='2016'").withColumn\
            ('사고대비 사망율',df_all['사망(명)']/df_all['사고(건)']*100).show()
end = time.time()
print(end-start)

+----------+--------+--------+--------+-------------------+----+------------------+
|      구분|사고(건)|사망(명)|부상(명)|               date|year|   사고대비 사망율|
+----------+--------+--------+--------+-------------------+----+------------------+
| 2016년1월|     192|       5|     387|2016-01-01 00:00:00|2016| 2.604166666666667|
| 2016년2월|     174|       6|     328|2016-02-01 00:00:00|2016|3.4482758620689653|
| 2016년3월|     217|       7|     435|2016-03-01 00:00:00|2016| 3.225806451612903|
| 2016년4월|     216|       7|     419|2016-04-01 00:00:00|2016|3.2407407407407405|
| 2016년5월|     239|      13|     522|2016-05-01 00:00:00|2016| 5.439330543933055|
| 2016년6월|     200|      12|     362|2016-06-01 00:00:00|2016|               6.0|
| 2016년7월|     227|       9|     460|2016-07-01 00:00:00|2016|3.9647577092511015|
| 2016년8월|     230|       7|     490|2016-08-01 00:00:00|2016|3.0434782608695654|
| 2016년9월|     187|      13|     347|2016-09-01 00:00:00|2016| 6.951871657754011|
|2016년10월|     183|      10|

### 5. 2015년 대비  사망이 가장 많이 증가한 2016년도 월을 구하시요

In [65]:
df_2016.withColumn('증가', df_2016['사망(명)']-df_2015['사망(명)']).show()

+----------+--------+--------+--------+-------------------+----+----+
|      구분|사고(건)|사망(명)|부상(명)|               date|year|증가|
+----------+--------+--------+--------+-------------------+----+----+
| 2016년1월|     192|       5|     387|2016-01-01 00:00:00|2016|   0|
| 2016년2월|     174|       6|     328|2016-02-01 00:00:00|2016|   0|
| 2016년3월|     217|       7|     435|2016-03-01 00:00:00|2016|   0|
| 2016년4월|     216|       7|     419|2016-04-01 00:00:00|2016|   0|
| 2016년5월|     239|      13|     522|2016-05-01 00:00:00|2016|   0|
| 2016년6월|     200|      12|     362|2016-06-01 00:00:00|2016|   0|
| 2016년7월|     227|       9|     460|2016-07-01 00:00:00|2016|   0|
| 2016년8월|     230|       7|     490|2016-08-01 00:00:00|2016|   0|
| 2016년9월|     187|      13|     347|2016-09-01 00:00:00|2016|   0|
|2016년10월|     183|      10|     367|2016-10-01 00:00:00|2016|   0|
|2016년11월|     200|       5|     418|2016-11-01 00:00:00|2016|   0|
|2016년12월|     168|       4|     364|2016-12-01 00:00:0